<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/src/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>python-awips: How to Access Data</h1>
<h3>Unidata AMS 2021 Student Conference</h3>

<div style="clear:both"></div>
</div>

---

### Focuses

* Cover the relevant methods for accessing EDEX and investigating what data is available.  
* This example we look at the "grid" data type and investigate the Global Forcast System (GFS) model.  
* We will talk quite a bit about the **DataAccessLayer** utility, and its [online documentation](http://unidata.github.io/python-awips/api/DataAccessLayer.html#) might be a helpful reference.

### Objectives

1. [Connect to EDEX](#1.-Connect-to-EDEX)
1. [Get a List of Supported Data Types](#2.-Get-a-List-of-Supported-Data-Types)
1. [Create a New Data Request and Set the Type](#3.-Create-a-New-Data-Request-and-Set-the-Type)
1. [Get Available Locations](#4.-Get-Available-Locations)
1. [Get Available Parameters](#5.-Get-Available-Parameters)
1. [Get Available Levels](#6.-Get-Available-Levels)
1. [Get Available Times](#7.-Get-Available-Times)
1. [Get the Data!](#8.-Get-the-Data!)

---

### Imports

Start by importing the DataAccessLayer package from python-awips:

In [ ]:
from awips.dataaccess import DataAccessLayer

---
## 1. Connect to EDEX

Define a url for your EDEX connection, and then point python-awips at that EDEX

In [ ]:
# Unidata's cloud EDEX instance is used in this example
edex_url = "edex-cloud.unidata.ucar.edu"
DataAccessLayer.changeEDEXHost(edex_url)

---
## 2. Get a List of Supported Data Types

[***DataAccessLayer.getSupportedDatatypes()***](http://unidata.github.io/python-awips/api/DataAccessLayer.html#awips.dataaccess.DataAccessLayer.getSupportedDatatypes) returns a list of supported data types offered by the EDEX server defined above. The code below shows how to populate, sort, and print out that list.

In [ ]:
dataTypes = DataAccessLayer.getSupportedDatatypes()
dataTypes.sort()
list(dataTypes)

<a href="#top">Top</a>

---
## 3. Create a New Data Request and Set the Type

Now create a new data request using [***DataAccessLayer.newDataRequest()***](http://unidata.github.io/python-awips/api/DataAccessLayer.html#awips.dataaccess.DataAccessLayer.newDataRequest), and set the data type using [***request.setDatatype()***](http://unidata.github.io/python-awips/api/IDataRequest.html#awips.dataaccess.IDataRequest.setDatatype).  Below we create a few different requests with different data types to show some differences with other methods.

For this example we are going to look at the "grid" data type, which is the model data can be found, along with some other datasets (such as MRMS).

In [ ]:
# Create a request for data type grid
grid_request = DataAccessLayer.newDataRequest()
grid_request.setDatatype("grid")

<a href="#top">Top</a>

---
## 4. Get Available Locations

Use the [***DataAccessLayer.getAvailableLocationNames(request)***](http://unidata.github.io/python-awips/api/DataAccessLayer.html#awips.dataaccess.DataAccessLayer.getAvailableLocationNames) method to find out what locations are available for the given dataset.  Typically these will be geographic locations or NWS sites, although in some instances it will be something else.  Take a look at what's outputted for the grid_request, for example.

In [ ]:
# Grid Locations
grid_locations = DataAccessLayer.getAvailableLocationNames(grid_request)
grid_locations.sort()
list(grid_locations)

<a href="#top">Top</a>

---
## 5. Get Available Parameters

We're setting the "location" (in this case, what model we are interested in) to specify our request before we look at the available parameters.

Take a look at the available parameters for the data set by using [***DataAccessLayer.getAvailableParameters(request)***](http://unidata.github.io/python-awips/api/DataAccessLayer.html#awips.dataaccess.DataAccessLayer.getAvailableParameters)

In [ ]:
# Pick a model and set the location for the grid request -- we'll be using the Global Forecast System (GFS)
grid_request.setLocationNames("GFS")
grid_params = DataAccessLayer.getAvailableParameters(grid_request)
grid_params.sort()
list(grid_params)

<a href="#top">Top</a>

---
## 6. Get Available Levels


Setting the parameters is just an option, you do not need to filter the data if you do not wish to.  Also, although we are only setting one parameter in this example, you can set multiple parameters by using an array:
```
params = ("param1", "param2", "param3"...)
request.setParameters(params)
```

Set a parameter, from the output above and take a look at what "levels" are available for the data set you're looking at using [***DataAccessLayer.getAvailableLevels(request)***](http://unidata.github.io/python-awips/api/DataAccessLayer.html#awips.dataaccess.DataAccessLayer.getAvailableLevels).

>Note: Not all datasets support levels.  If you are trying this with another dataset and run into an exception (error), it's most likely because levels are not supported for that data type.

In [ ]:
# For grid data we'll use the temperature parameter ("T")
grid_request.setParameters("T")
grid_levels = DataAccessLayer.getAvailableLevels(grid_request)
for lvl in grid_levels:
    print(lvl)

* **0.0SFC** is the Surface level
* **FHAG** stands for Fixed Height Above Ground (in meters)
* **NTAT** stands for Nominal Top of the ATmosphere
* **BL** stands for Boundary Layer, where **0.0_30.0BL** reads as *0-30 mb above ground level*  
* **TROP** is the Tropopause level

In [ ]:
# We'll set the level to surface level
grid_request.setLevels("0.0SFC")

<a href="#top">Top</a>

---
## 7. Get Available Times

Take a look at what time options are available for the data you're looking at using the [***DataAccessLayer.getAvailableTimes()***](http://unidata.github.io/python-awips/api/DataAccessLayer.html#awips.dataaccess.DataAccessLayer.getAvailableTimes) method:

* **getAvailableTimes(request, True)** will return an object of *run times* - formatted as `YYYY-MM-DD HH:MM:SS`
* **getAvailableTimes(request)** will return an object of all times - formatted as `YYYY-MM-DD HH:MM:SS (F:ff)`
* **getForecastRun(cycle, times)** will return a DataTime array for a single forecast cycle.

In [ ]:
# Available grid times
grid_cycles = DataAccessLayer.getAvailableTimes(grid_request, True)
grid_times = DataAccessLayer.getAvailableTimes(grid_request)
## Using -1 in an array will access the last element of the array 
##  (using -2 will access the second last element, and so on)
grid_fcstRun = DataAccessLayer.getForecastRun(grid_cycles[-1], grid_times)

## print out the time instances
times = []
for fcst in grid_fcstRun:
    print(fcst.getRefTime(), ':', fcst.getFcstTime(), 'seconds')

<a href="#top">Top</a>

---
## 8. Get the Data!

Now that we have our `request` and DataTime `fcstRun` arrays ready, it's time to request the data array from EDEX.  Depending on what kind of data we're working with, we'll either use [***DataAccessLayer.getGridData()***](http://unidata.github.io/python-awips/api/DataAccessLayer.html#awips.dataaccess.DataAccessLayer.getGridData) or [***DataAccessLayer.getGeometryData()***](http://unidata.github.io/python-awips/api/DataAccessLayer.html#awips.dataaccess.DataAccessLayer.getGeometryData)

> Note: We have more, detailed notebooks about how analyze and visualize the data once you have what you want.

In [ ]:
## Grid Data
grid_response = DataAccessLayer.getGridData(grid_request, [grid_fcstRun[-1]])
for grid in grid_response:
    grid_data = grid.getRawData()
    lons, lats = grid.getLatLonCoords()
    print('Time :', grid.getDataTime(), "-", grid.getDataTime().getFcstTime(), 'seconds')

## Take a look at some information in our data
print('Model:', str(grid.getLocationName()))
print('Parm :', str(grid.getParameter()))
print('Unit :', str(grid.getUnit()))
print(grid_data.shape)

<a href="#top">Top</a>

---

## See also

Several functions are used throughout this notebook from the DataAccessLayer class in python-awips, to see full documentation for these functions vist [**here**](http://unidata.github.io/python-awips/api/DataAccessLayer.html#).

### Related Notebooks

* [python-awips: Working with Surface Obs](https://nbviewer.jupyter.org/github/Unidata/pyaos-ams-2021/blob/master/notebooks/visualization/python-awips-WorkingWithSurfaceObs.ipynb)
* [python-awips: Working with Models](https://nbviewer.jupyter.org/github/Unidata/pyaos-ams-2021/blob/master/notebooks/visualization/python-awips-WorkingWithModels.ipynb)
* [python-awips: Working with Satellite Data](https://nbviewer.jupyter.org/github/Unidata/pyaos-ams-2021/blob/master/notebooks/visualization/python-awips-WorkingWithSatelliteData.ipynb)
* [python-awips: Working with Upper Air Obs](https://nbviewer.jupyter.org/github/Unidata/pyaos-ams-2021/blob/master/notebooks/visualization/python-awips-WorkingWithUpperAirObs.ipynb)
* [python-awips: Working with Maps and Topography Databases](https://nbviewer.jupyter.org/github/Unidata/pyaos-ams-2021/blob/master/notebooks/visualization/python-awips-WorkingWithMapsTopoDatabases.ipynb)

<a href="#top">Top</a>